Blatt 6 - Praktische Aufgabe

In [58]:
import numpy as np
import math
import matplotlib.pyplot


a)

Zur einfacheren Berechnung der Matrix splitten wir die linke Seite von (3) in drei Teile:

$o=h_{i}a_{2}^{(i-1)}$
$p=2(h_{i}+h_{i+1})a_{2}^{(i)}$
$q=h_{i+1}a_{2}^{(i+1)}$

Sodass o+p+q = ... (3)




In [59]:
def compute_rhs(x,y):
    b=np.zeros((len(x)-2))#1,..,n-1
    h=np.diff(x)
    for j in range(len(b)):
        i=j+1
        #Da b und h hier anders indiziert sind als y verwenden wir für b und h index j und für y index i
        b[j]=3*(((y[i+1]-y[i])/(h[j+1]))-((y[i]-y[i-1])/(h[j])))
    return b

def compute_matrix(x):
    n = len(x)
    h=np.diff(x)
    for i in range(1,n-1):
        #Indieces von h wurden wieder angepasst
        if (i-1==0):
            o=0
        else:
            o=h[i-1]


    


x=np.array([0,1,2,3])
y=np.array([0,1,0,1])
#A=compute_matrix(x)
b=compute_rhs(x,y)

[-6.  6.]


b)

In [60]:
def compute_coeff(x,y):
    return 1,2,3,4

a0,a1,a2,a3 = compute_coeff(x,y)